In [1]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
    IS_COLAB = True
except Exception:
    IS_COLAB = False

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

if not tf.test.is_gpu_available():
    print("No GPU was detected. CNNs can be very slow without a GPU.")
    if IS_COLAB:
        print("Go to Runtime > Change runtime and select a GPU hardware accelerator.")

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)
tf.random.set_seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "cnn"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)
    

def plot_image(image):
    plt.imshow(image, cmap="gray", interpolation="nearest")
    plt.axis("off")

def plot_color_image(image):
    plt.imshow(image, interpolation="nearest")
    plt.axis("off")

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
No GPU was detected. CNNs can be very slow without a GPU.


In [3]:
default_reber_grammar = [
    [("B", 1)],           # (state 0) =B=>(state 1)
    [("T", 2), ("P", 3)], # (state 1) =T=>(state 2) or =P=>(state 3)
    [("S", 2), ("X", 4)], # (state 2) =S=>(state 2) or =X=>(state 4)
    [("T", 3), ("V", 5)], # and so on...
    [("X", 3), ("S", 6)],
    [("P", 4), ("V", 6)],
    [("E", None)]]        # (state 6) =E=>(terminal state)

embedded_reber_grammar = [
    [("B", 1)],
    [("T", 2), ("P", 3)],
    [(default_reber_grammar, 4)],
    [(default_reber_grammar, 5)],
    [("T", 6)],
    [("P", 6)],
    [("E", None)]]

def generate_string(grammar):
    state = 0
    output = []
    while state is not None:
        index = np.random.randint(len(grammar[state]))
        production, state = grammar[state][index]
        if isinstance(production, list):
            production = generate_string(grammar=production)
        output.append(production)
    return "".join(output)

In [4]:
POSSIBLE_CHARS = "BEPSTVX"

def generate_corrupted_string(grammar, chars=POSSIBLE_CHARS):
    good_string = generate_string(grammar)
    index = np.random.randint(len(good_string))
    good_char = good_string[index]
    bad_char = np.random.choice(sorted(set(chars) - set(good_char)))
    return good_string[:index] + bad_char + good_string[index + 1:]

In [7]:
def string_to_ids(s, chars=POSSIBLE_CHARS):
    return [POSSIBLE_CHARS.index(c) for c in s]

In [5]:
def generate_dataset(size):
    good_strings = [string_to_ids(generate_string(embedded_reber_grammar))
                    for _ in range(size // 2)]
    bad_strings = [string_to_ids(generate_corrupted_string(embedded_reber_grammar))
                   for _ in range(size - size // 2)]
    all_strings = good_strings + bad_strings
    X = tf.ragged.constant(all_strings, ragged_rank=1)
    y = np.array([[1.] for _ in range(len(good_strings))] +
                 [[0.] for _ in range(len(bad_strings))])
    return X, y

In [8]:

np.random.seed(42)

X_train, y_train = generate_dataset(10000)
X_valid, y_valid = generate_dataset(2000)

In [16]:

embedding_size = 5

model = keras.models.Sequential([
    keras.layers.InputLayer(input_shape=[None], dtype=tf.int32, ragged=True),
    keras.layers.Embedding(input_dim=len(POSSIBLE_CHARS), output_dim=embedding_size),
    keras.layers.GRU(30),
    keras.layers.Dense(1, activation="sigmoid")
])
optimizer = keras.optimizers.SGD(lr=0.02, momentum = 0.95, nesterov=True)
model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_valid, y_valid))

Epoch 1/20


C:\Users\ikuchin\AppData\Local\Continuum\miniconda3\envs\tf22\lib\site-packages\tensorflow\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(


313/313 [==============================] - 6s 20ms/step - loss: 0.6924 - accuracy: 0.5155 - val_loss: 0.6867 - val_accuracy: 0.5380
Epoch 2/20
313/313 [==============================] - 5s 16ms/step - loss: 0.6675 - accuracy: 0.5639 - val_loss: 0.6567 - val_accuracy: 0.6130
Epoch 3/20
313/313 [==============================] - 5s 15ms/step - loss: 0.6494 - accuracy: 0.5922 - val_loss: 0.6440 - val_accuracy: 0.6500
Epoch 4/20
313/313 [==============================] - 5s 15ms/step - loss: 0.6377 - accuracy: 0.5989 - val_loss: 0.6279 - val_accuracy: 0.6000acy: 
Epoch 5/20
313/313 [==============================] - 5s 15ms/step - loss: 0.6010 - accuracy: 0.6474 - val_loss: 0.5722 - val_accuracy: 0.7410
Epoch 6/20
313/313 [==============================] - 5s 15ms/step - loss: 0.5419 - accuracy: 0.7098 - val_loss: 0.5138 - val_accuracy: 0.7225
Epoch 7/20
313/313 [==============================] - 5s 15ms/step - loss: 0.4545 - accuracy: 0.7809 - val_loss: 0.3738 - val_accuracy: 0.8670
Epoch

In [15]:
X_train

<tf.RaggedTensor [[0, 4, 0, 2, 4, 4, 4, 5, 2, 6, 4, 5, 2, 6, 4, 4, 5, 2, 3, 1, 4, 1], [0, 2, 0, 2, 4, 5, 2, 3, 1, 2, 1], [0, 2, 0, 2, 5, 5, 1, 2, 1], [0, 2, 0, 2, 5, 2, 6, 5, 5, 1, 2, 1], [0, 2, 0, 4, 6, 6, 4, 4, 4, 4, 5, 5, 1, 2, 1], [0, 2, 0, 2, 5, 2, 3, 1, 2, 1], [0, 2, 0, 4, 6, 6, 5, 2, 3, 1, 2, 1], [0, 2, 0, 4, 3, 3, 3, 3, 3, 3, 3, 6, 3, 1, 2, 1], [0, 4, 0, 2, 5, 5, 1, 4, 1], [0, 2, 0, 4, 6, 6, 5, 5, 1, 2, 1], [0, 2, 0, 4, 6, 6, 5, 2, 3, 1, 2, 1], [0, 4, 0, 4, 6, 6, 5, 5, 1, 4, 1], [0, 2, 0, 2, 5, 5, 1, 2, 1], [0, 2, 0, 2, 5, 5, 1, 2, 1], [0, 2, 0, 4, 3, 6, 3, 1, 2, 1], [0, 2, 0, 2, 5, 5, 1, 2, 1], [0, 2, 0, 2, 4, 5, 2, 3, 1, 2, 1], [0, 2, 0, 4, 6, 6, 5, 5, 1, 2, 1], [0, 4, 0, 2, 4, 5, 2, 6, 5, 5, 1, 4, 1], [0, 4, 0, 2, 5, 5, 1, 4, 1], [0, 4, 0, 4, 3, 3, 3, 3, 3, 3, 3, 6, 6, 5, 5, 1, 4, 1], [0, 2, 0, 4, 3, 3, 3, 6, 6, 4, 4, 4, 4, 5, 2, 3, 1, 2, 1], [0, 4, 0, 2, 4, 4, 5, 5, 1, 4, 1], [0, 2, 0, 4, 6, 6, 4, 5, 5, 1, 2, 1], [0, 4, 0, 4, 6, 3, 1, 4, 1], [0, 2, 0, 4, 3, 3, 3, 3, 3, 6, 6